In [1]:
import json
import pandas as pd
from a2ginputstream import A2GClient
import random
import time
import datetime
import logging

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

c:\Users\DaniloAraneda\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\DaniloAraneda\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.fb5ae2tyxyh2ijrdkgdgq3xbklktf43h.gfortran-win_amd64.dll
c:\Users\DaniloAraneda\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


mongo_conn: None, mongo_db: None from cloud_inputstream.py
mongo_conn: None, mongo_db: None from init.py


## read data

In [2]:
print("reading data...")

print ("reading users...")
users_str = open("../data/raw/mock_users.json", "r").read()
users = json.loads(users_str)

print ("reading machines...")
machines_str = open("../data/raw/mock_machines.json", "r").read()
machines = json.loads(machines_str)


reading data...
reading users...
reading machines...


In [3]:
df_users = pd.DataFrame(users)
df_users.head()

,name,role
0,John Doe,admin
1,Jane Doe,user
2,John Smith,user
3,Smith Alfred,user
4,Richard Alfredo,user


## merge data

In [4]:
import datetime


print("merging data...")

# 70, 25, 25%
def get_status():
    r = random.randint(0, 100)
    if r < 50: return "active"
    elif r < 75: return "inactive"
    else: return "error"

df_users["machines"] = df_users["name"].apply(lambda _: machines[random.randint(0, len(machines)-1)])
df_users["status"] = df_users["name"].apply(lambda _: get_status())
df_users["ton"] = df_users["machines"].apply(lambda _: random.randint(0, 100))
df_users["start_on"] = df_users["machines"].apply(lambda _: (datetime.datetime.utcnow() - datetime.timedelta(minutes=5)).isoformat())
df_users["end_on"] = df_users["machines"].apply(lambda _: datetime.datetime.utcnow().isoformat())
print("done merging data")

merging data...
done merging data


In [5]:
df_users.head()

,name,role,machines,status,ton,start_on,end_on
0,John Doe,admin,C303,active,96,2024-10-29T11:36:09.452580,2024-10-29T11:41:09.453093
1,Jane Doe,user,C301,active,21,2024-10-29T11:36:09.452580,2024-10-29T11:41:09.453093
2,John Smith,user,C303,error,69,2024-10-29T11:36:09.452580,2024-10-29T11:41:09.453093
3,Smith Alfred,user,C303,error,59,2024-10-29T11:36:09.452580,2024-10-29T11:41:09.453093
4,Richard Alfredo,user,C303,inactive,0,2024-10-29T11:36:09.452580,2024-10-29T11:41:09.453093


## simulate error ETL and long time process

In [6]:
rand_error = random.randint(0, 100)
if rand_error < 30:
    print("error")
    raise Exception("Error")

rand_ltp = random.randint(0, 100)
if rand_ltp < 30:
    for _ in range(0, 500):
        time.sleep(1) 

## insert data

In [7]:
print("sending data...")
USER_PAT = "8f8ff571c7344ce9b6fae09f1ff9c5b1"
IKEY = "1c01a94bea5948faabda"
client = A2GClient(token=USER_PAT)
client.insert_data(IKEY, df_users.to_dict("records"))



2024-10-29 08:41:09,526 - DEBUG - Starting new HTTPS connection (1): v2apigateway.a2g.io:443


sending data...


2024-10-29 08:41:09,942 - DEBUG - https://v2apigateway.a2g.io:443 "GET /Inputstream/Ikey/1c01a94bea5948faabda HTTP/1.1" 200 None
2024-10-29 08:41:09,952 - INFO - Getting inputstream 1c01a94bea5948faabda from A2G...
2024-10-29 08:41:09,955 - WARNING - Inputstream discovered
2024-10-29 08:41:09,964 - DEBUG - Starting new HTTPS connection (1): v2streams.a2g.io:443
2024-10-29 08:41:10,962 - DEBUG - https://v2streams.a2g.io:443 "POST /Data/Insert HTTP/1.1" 200 None
2024-10-29 08:41:10,969 - INFO - batch 1, docs: [0 - 31] - 200 - {"message":"Data Received Success, DataToy inputstream","data":null,"success":true,"errors":null}
